In [1]:
import yaml

with open("elasticsearch_example.yml") as f:
    data = yaml.load(f,Loader=yaml.FullLoader)
    print(data)

{'cluster.name': 'od-fts1', 'node.name': 'od-fts1a', 'node.master': True, 'node.data': True, 'index.number_of_shards': 2, 'index.number_of_replicas': 1, 'bootstrap.mlockall': True, 'gateway.recover_after_nodes': 1, 'gateway.recover_after_time': '10m', 'gateway.expected_nodes': 2, 'action.disable_close_all_indices': True, 'action.disable_delete_all_indices': True, 'action.disable_shutdown': True, 'indices.recovery.max_bytes_per_sec': '100mb'}


In [11]:
with open("elasticsearch_example.yml") as f:
    data =  yaml.load(f,Loader=yaml.FullLoader)
    print(yaml.dump(data))

action.disable_close_all_indices: true
action.disable_delete_all_indices: true
action.disable_shutdown: true
bootstrap.mlockall: true
cluster.name: od-fts1
gateway:
  expected_nodes: 2
gateway.recover_after_nodes: 1
gateway.recover_after_time: 10m
index.number_of_replicas: 1
index.number_of_shards: 2
indices.recovery.max_bytes_per_sec: 100mb
node.data: true
node.master: true
node.name: od-fts1a



### Flattening dictionary using recursive function

In [13]:
from collections.abc import MutableMapping

def flatten_dict(d: MutableMapping, parent_key:str ="",sep: str=".") -> MutableMapping:
    items=[]
    for k ,v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v,MutableMapping):
            items.extend(flatten_dict(v,new_key,sep=sep).items()) #return value -> tuple
        else:
            items.append((new_key,v)) #add tuple
    return dict(items)  #convert tuple into key:value

In [15]:
with open("elasticsearch_example.yml") as f:
    data = yaml.load(f,Loader=yaml.FullLoader)
    dat=flatten_dict(data)
    print(dat)

{'cluster.name': 'od-fts1', 'node.name': 'od-fts1a', 'node.master': True, 'node.data': True, 'index.number_of_shards': 2, 'index.number_of_replicas': 1, 'bootstrap.mlockall': True, 'gateway.recover_after_nodes': 1, 'gateway.recover_after_time': '10m', 'gateway.expected_nodes': 2, 'action.disable_close_all_indices': True, 'action.disable_delete_all_indices': True, 'action.disable_shutdown': True, 'indices.recovery.max_bytes_per_sec': '100mb'}


In [16]:
with open("elasticsearch_example.yml") as f:
    data = yaml.load(f,Loader=yaml.FullLoader)
    dat=flatten_dict(data)
    print(yaml.dump(dat))

action.disable_close_all_indices: true
action.disable_delete_all_indices: true
action.disable_shutdown: true
bootstrap.mlockall: true
cluster.name: od-fts1
gateway.expected_nodes: 2
gateway.recover_after_nodes: 1
gateway.recover_after_time: 10m
index.number_of_replicas: 1
index.number_of_shards: 2
indices.recovery.max_bytes_per_sec: 100mb
node.data: true
node.master: true
node.name: od-fts1a



#### performance Benchmark

In [17]:
%timeit flatten_dict({'a': 1, 'c': {'a': 2, 'b': {'x': 3, 'y': 4, 'z': 5}}, 'd': [6, 7, 8]})

5.52 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Flattening a Dict using recursive function + generators

A much better solution is to use Python's generators which is an object that can pause execution and remembers the state that can be resumed later. By using a generator, we can get rid of the temporary list with no change in behavior.

In [20]:
from collections.abc import MutableMapping

def _flatten_dict_gen(d:dict,parent_key,sep):
    for k ,v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v,MutableMapping):
            yield from flatten_dict(v,new_key,sep=sep).items()
        else:
            yield new_key,v
            
def flatten_dict(d:MutableMapping, parent_key:str ="",sep:str="."):
    return dict(_flatten_dict_gen(d,parent_key,sep))

flatten_dict({'a': 1, 'c': {'a': 2, 'b': {'x': 3, 'y': 4, 'z': 5}}, 'd': [6, 7, 8]})

{'a': 1, 'c.a': 2, 'c.b.x': 3, 'c.b.y': 4, 'c.b.z': 5, 'd': [6, 7, 8]}

In [21]:
%timeit flatten_dict({'a': 1, 'c': {'a': 2, 'b': {'x': 3, 'y': 4, 'z': 5}}, 'd': [6, 7, 8]})

5.76 µs ± 62.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
